In [1]:
import pickle
import logging
import datetime
import sys
sys.path.append('..')

import numpy as np

from mod.Demand import Demand
from mod.SimSale import SimSale

### 对于参数估计效果的影响的因素（固定三个属性及三个水平）
- 销售额 qty 10~1000
- 样本量 n 10~1000
- 商品种类缺失的数量（随机） rlack 1~26

In [2]:
# 固定三种属性及每种属性下水平的个数
level_size = [3,3,3]
goods_attr={'A': ['a', 'b','c'], 'B': ['a', 'b','c'], 'C':['a','b','c']}

# 参数数量
qty_list   = np.arange(start=100,stop=1000,step=100)
n_list     = np.arange(start=10,stop=1000,step=50)
rlack_list = np.arange(start=0.1,stop=0.9,step=0.1)

param_grid = np.meshgrid(qty_list,n_list,rlack_list)
param_grid = [grid.flatten() for grid in param_grid]

In [3]:
sim_sale = SimSale(level_size=level_size,seed=123)
logging.basicConfig(filename='logging.log',encoding='utf-8',level=logging.INFO)

score_list = []
for i in range(len(param_grid[0])):
    qty_i        = param_grid[0][i]
    n_i          = param_grid[1][i]
    rlack_list_i = param_grid[2][i]
    
    sim_data = sim_sale.generate_sale(lam=qty_i,size=n_i,un_ava_frac=rlack_list_i,un_ava_mix=False)
    
    mod = Demand(data=sim_data,goods_attr=goods_attr)
    mod.fit()
    
    score = mod.score(SimSale=sim_sale)
    score_list.append(score)
    
    log_info = 'Time:{} | Finish:{} | qty:{} | n:{} | rlack_list_i:{} | score:{}'.format(
        datetime.datetime.now(),
        str(round(i/len(param_grid[0])*100,2))+'%',
        qty_i,
        n_i,
        round(rlack_list_i,1),
        round(score,4)
    )
    logging.info(log_info)
pickle.dump(obj=score_list,file=open(file='score_list',mode='wb'))

KeyboardInterrupt: 